# Conversation Service

In [1]:
import json
import sys
import os
sys.path.append(os.path.join(os.getcwd(),'..','..'))
import watson_developer_cloud

In [2]:
USERNAME = os.environ.get('CONVERSATION_USERNAME','<USERNAME>')
PASSWORD = os.environ.get('CONVERSATION_PASSWORD','<PASSWORD>')

In [3]:
conversation = watson_developer_cloud.ConversationV1(username=USERNAME,
                                                     password=PASSWORD,
                                                     version='2016-09-20')

## Pizza Chatbot

Let's create and chat with a simple pizza bot. We'll start by defining the bot's workspace, then add intents and examples to recognize pizza orders. Once the chatbot is configured, we'll send a message to converse with our pizza bot.

In [4]:
# define the dialog for our example workspace
dialog_nodes = [{'conditions': '#order_pizza',
                 'context': None,
                 'description': None,
                 'dialog_node': 'YesYouCan',
                 'go_to': None,
                 'metadata': None,
                 'output': {'text': {'selection_policy': 'random',
                                     'values': ['Yes You can!', 'Of course!']}},
                 'parent': None,
                 'previous_sibling': None,}]

# create an example workspace
workspace = conversation.create_workspace(name='example_workspace',
                                          description='An example workspace.',
                                          language='en',
                                          dialog_nodes=dialog_nodes)

# print response and save workspace_id
print(json.dumps(workspace, indent=2))
workspace_id=workspace['workspace_id']

{
  "name": "example_workspace",
  "created": "2017-04-17T18:55:39.740Z",
  "updated": "2017-04-17T18:55:39.740Z",
  "language": "en",
  "metadata": null,
  "description": "An example workspace.",
  "workspace_id": "0247c2f5-4155-4be5-bd51-167de32f2f39"
}


In [5]:
# add an intent to the workspace
intent = conversation.create_intent(workspace_id=workspace_id,
                                    intent='order_pizza',
                                    description='A pizza order.')
print(json.dumps(intent, indent=2))

{
  "intent": "order_pizza",
  "created": "2017-04-17T18:55:40.899Z",
  "updated": "2017-04-17T18:55:40.899Z",
  "description": "A pizza order."
}


In [6]:
# add examples to the intent
example1 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Can I order a pizza?')
example2 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='I want to order a pizza.')
example3 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='pizza order')
example4 = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='pizza to go')

print(json.dumps(example1, indent=2))
print(json.dumps(example2, indent=2))
print(json.dumps(example3, indent=2))
print(json.dumps(example4, indent=2))

{
  "text": "Can I order a pizza?",
  "created": "2017-04-17T18:55:41.755Z",
  "updated": "2017-04-17T18:55:41.755Z"
}
{
  "text": "I want to order a pizza.",
  "created": "2017-04-17T18:55:42.172Z",
  "updated": "2017-04-17T18:55:42.172Z"
}
{
  "text": "pizza order",
  "created": "2017-04-17T18:55:42.606Z",
  "updated": "2017-04-17T18:55:42.606Z"
}
{
  "text": "pizza to go",
  "created": "2017-04-17T18:55:43.018Z",
  "updated": "2017-04-17T18:55:43.018Z"
}


In [9]:
# check workspace status (wait for training to complete)
workspace = conversation.get_workspace(workspace_id=workspace_id)
print('The workspace status is: {0}'.format(workspace['status']))
if workspace['status'] == 'Available':
    print('Ready to chat!')
else:
    print('The workspace should be available shortly. Please try again in 30s.')
    print('(You can send messages, but not all functionality will be supported yet.)')

The workspace status is: Available
Ready to chat!


In [10]:
# start a chat with the pizza bot
message_input = {'text': 'Can I order a pizza?'}
response = conversation.message(workspace_id=workspace_id,
                                message_input=message_input)
print(json.dumps(response, indent=2))

{
  "intents": [
    {
      "intent": "order_pizza",
      "confidence": 1
    }
  ],
  "entities": [],
  "input": {
    "text": "Can I order a pizza?"
  },
  "output": {
    "log_messages": [],
    "text": [
      "Yes You can!"
    ],
    "nodes_visited": [
      "YesYouCan"
    ]
  },
  "context": {
    "conversation_id": "a84cbf65-d41d-4092-8450-e03c986a3340",
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "dialog_turn_counter": 1,
      "dialog_request_counter": 1,
      "_node_output_map": {
        "YesYouCan": [
          0,
          0,
          1
        ]
      },
      "branch_exited": true,
      "branch_exited_reason": "completed"
    }
  }
}


In [11]:
# continue a chat with the pizza bot
# (when you send multiple requests for the same conversation,
#  then include the context object from the previous response)
message_input = {'text': 'medium'}
response = conversation.message(workspace_id=workspace_id,
                                message_input=message_input,
                                context=response['context'])
print(json.dumps(response, indent=2))

{
  "intents": [
    {
      "intent": "order_pizza",
      "confidence": 1
    }
  ],
  "entities": [],
  "input": {
    "text": "medium"
  },
  "output": {
    "log_messages": [],
    "text": [
      "Of course!"
    ],
    "nodes_visited": [
      "YesYouCan"
    ]
  },
  "context": {
    "conversation_id": "a84cbf65-d41d-4092-8450-e03c986a3340",
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "dialog_turn_counter": 2,
      "dialog_request_counter": 2,
      "_node_output_map": {
        "YesYouCan": [
          1,
          0,
          1
        ]
      },
      "branch_exited": true,
      "branch_exited_reason": "completed"
    }
  }
}


## Operation Examples

The following examples demonstrate each operation of the Conversation service. They use the pizza chatbot workspace configured above.

### Message

In [12]:
message_input = {'text': 'Can I order a pizza?'}
response = conversation.message(workspace_id=workspace_id,
                                message_input=message_input)
print(json.dumps(response, indent=2))

{
  "intents": [
    {
      "intent": "order_pizza",
      "confidence": 1
    }
  ],
  "entities": [],
  "input": {
    "text": "Can I order a pizza?"
  },
  "output": {
    "log_messages": [],
    "text": [
      "Of course!"
    ],
    "nodes_visited": [
      "YesYouCan"
    ]
  },
  "context": {
    "conversation_id": "bceff6eb-f89b-48d3-9190-99e9cca6a3e7",
    "system": {
      "dialog_stack": [
        {
          "dialog_node": "root"
        }
      ],
      "dialog_turn_counter": 1,
      "dialog_request_counter": 1,
      "_node_output_map": {
        "YesYouCan": [
          0,
          1,
          0
        ]
      },
      "branch_exited": true,
      "branch_exited_reason": "completed"
    }
  }
}


### Workspaces

In [13]:
response = conversation.create_workspace(name='test_workspace',
                                         description='Test workspace.',
                                         language='en',
                                         metadata={})
print(json.dumps(response, indent=2))
test_workspace_id = response['workspace_id']

{
  "name": "test_workspace",
  "created": "2017-04-17T18:57:41.168Z",
  "updated": "2017-04-17T18:57:41.168Z",
  "language": "en",
  "metadata": {},
  "description": "Test workspace.",
  "workspace_id": "d8cc32e4-5fb4-47ec-944c-0d7fcd59651c"
}


In [14]:
response = conversation.delete_workspace(workspace_id=test_workspace_id)
print(json.dumps(response, indent=2))

{}


In [15]:
response = conversation.get_workspace(workspace_id=workspace_id, export=True)
print(json.dumps(response, indent=2))

{
  "name": "example_workspace",
  "created": "2017-04-17T18:55:39.740Z",
  "intents": [
    {
      "intent": "order_pizza",
      "created": "2017-04-17T18:55:40.899Z",
      "updated": "2017-04-17T18:55:43.018Z",
      "examples": [
        {
          "text": "Can I order a pizza?",
          "created": "2017-04-17T18:55:41.755Z",
          "updated": "2017-04-17T18:55:41.755Z"
        },
        {
          "text": "I want to order a pizza.",
          "created": "2017-04-17T18:55:42.172Z",
          "updated": "2017-04-17T18:55:42.172Z"
        },
        {
          "text": "pizza order",
          "created": "2017-04-17T18:55:42.606Z",
          "updated": "2017-04-17T18:55:42.606Z"
        },
        {
          "text": "pizza to go",
          "created": "2017-04-17T18:55:43.018Z",
          "updated": "2017-04-17T18:55:43.018Z"
        }
      ],
      "description": "A pizza order."
    }
  ],
  "updated": "2017-04-17T18:55:43.018Z",
  "entities": [],
  "language": "en",
  

In [16]:
response = conversation.list_workspaces()
print(json.dumps(response, indent=2))

{
  "workspaces": [
    {
      "name": "Car_Dashboard",
      "created": "2016-07-19T16:31:17.236Z",
      "updated": "2017-04-11T02:29:46.290Z",
      "language": "en",
      "metadata": {
        "runtime_version": "2016-09-20"
      },
      "description": "Cognitive Car workspace which allows multi-turn conversations to perform tasks in the car.",
      "workspace_id": "8d869397-411b-4f0a-864d-a2ba419bb249"
    },
    {
      "name": "example_workspace",
      "created": "2017-04-17T18:07:47.842Z",
      "updated": "2017-04-17T18:07:47.842Z",
      "language": "en",
      "metadata": null,
      "description": "An example workspace.",
      "workspace_id": "a7196fc9-c458-4b75-963c-dc4a3dc32ef7"
    },
    {
      "name": "example_workspace",
      "created": "2017-04-17T18:42:40.986Z",
      "updated": "2017-04-17T18:54:21.407Z",
      "language": "en",
      "metadata": null,
      "description": "An example workspace for ordering pizza.",
      "workspace_id": "88a4327b-421c-436

In [17]:
response = conversation.update_workspace(workspace_id=workspace_id,
                                         description='An example workspace for ordering pizza.')
print(json.dumps(response, indent=2))

{
  "name": "example_workspace",
  "created": "2017-04-17T18:55:39.740Z",
  "updated": "2017-04-17T18:57:45.626Z",
  "language": "en",
  "metadata": null,
  "description": "An example workspace for ordering pizza.",
  "workspace_id": "0247c2f5-4155-4be5-bd51-167de32f2f39"
}


### Intents

In [18]:
response = conversation.create_intent(workspace_id=workspace_id,
                                      intent='test_intent',
                                      description='Test intent.')
print(json.dumps(response, indent=2))

{
  "intent": "test_intent",
  "created": "2017-04-17T18:57:47.513Z",
  "updated": "2017-04-17T18:57:47.513Z",
  "description": "Test intent."
}


In [19]:
response = conversation.delete_intent(workspace_id=workspace_id,
                                      intent='test_intent')
print(json.dumps(response, indent=2))

{}


In [20]:
response = conversation.get_intent(workspace_id=workspace_id,
                                   intent='order_pizza',
                                   export=True)
print(json.dumps(response, indent=2))

{
  "intent": "order_pizza",
  "created": "2017-04-17T18:55:40.899Z",
  "updated": "2017-04-17T18:55:43.018Z",
  "examples": [
    {
      "text": "Can I order a pizza?",
      "created": "2017-04-17T18:55:41.755Z",
      "updated": "2017-04-17T18:55:41.755Z"
    },
    {
      "text": "I want to order a pizza.",
      "created": "2017-04-17T18:55:42.172Z",
      "updated": "2017-04-17T18:55:42.172Z"
    },
    {
      "text": "pizza order",
      "created": "2017-04-17T18:55:42.606Z",
      "updated": "2017-04-17T18:55:42.606Z"
    },
    {
      "text": "pizza to go",
      "created": "2017-04-17T18:55:43.018Z",
      "updated": "2017-04-17T18:55:43.018Z"
    }
  ],
  "description": "A pizza order."
}


In [21]:
response = conversation.list_intents(workspace_id=workspace_id,
                                     export=True)
print(json.dumps(response, indent=2))

{
  "intents": [
    {
      "intent": "order_pizza",
      "created": "2017-04-17T18:55:40.899Z",
      "updated": "2017-04-17T18:55:43.018Z",
      "examples": [
        {
          "text": "Can I order a pizza?",
          "created": "2017-04-17T18:55:41.755Z",
          "updated": "2017-04-17T18:55:41.755Z"
        },
        {
          "text": "I want to order a pizza.",
          "created": "2017-04-17T18:55:42.172Z",
          "updated": "2017-04-17T18:55:42.172Z"
        },
        {
          "text": "pizza order",
          "created": "2017-04-17T18:55:42.606Z",
          "updated": "2017-04-17T18:55:42.606Z"
        },
        {
          "text": "pizza to go",
          "created": "2017-04-17T18:55:43.018Z",
          "updated": "2017-04-17T18:55:43.018Z"
        }
      ],
      "description": "A pizza order."
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/0247c2f5-4155-4be5-bd51-167de32f2f39/intents?version=2016-09-20&export=true"
  }
}


In [22]:
response = conversation.update_intent(workspace_id=workspace_id,
                                      intent='order_pizza',
                                      new_intent='order_pizza',
                                      new_description='Order a pizza.')
print(json.dumps(response, indent=2))

{
  "intent": "order_pizza",
  "created": "2017-04-17T18:55:40.899Z",
  "updated": "2017-04-17T18:57:52.129Z",
  "description": "Order a pizza."
}


### Examples

In [23]:
response = conversation.create_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Gimme a pizza with pepperoni')
print(json.dumps(response, indent=2))

{
  "text": "Gimme a pizza with pepperoni",
  "created": "2017-04-17T18:57:54.817Z",
  "updated": "2017-04-17T18:57:54.817Z"
}


In [24]:
response = conversation.delete_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='Gimme a pizza with pepperoni')
print(json.dumps(response, indent=2))

{}


In [25]:
response = conversation.get_example(workspace_id=workspace_id,
                                    intent='order_pizza',
                                    text='I want to order a pizza.')
print(json.dumps(response, indent=2))

{
  "text": "I want to order a pizza.",
  "created": "2017-04-17T18:55:42.172Z",
  "updated": "2017-04-17T18:55:42.172Z"
}


In [26]:
response = conversation.list_examples(workspace_id=workspace_id,
                                      intent='order_pizza')
print(json.dumps(response, indent=2))

{
  "examples": [
    {
      "text": "Can I order a pizza?",
      "created": "2017-04-17T18:55:41.755Z",
      "updated": "2017-04-17T18:55:41.755Z"
    },
    {
      "text": "I want to order a pizza.",
      "created": "2017-04-17T18:55:42.172Z",
      "updated": "2017-04-17T18:55:42.172Z"
    },
    {
      "text": "pizza order",
      "created": "2017-04-17T18:55:42.606Z",
      "updated": "2017-04-17T18:55:42.606Z"
    },
    {
      "text": "pizza to go",
      "created": "2017-04-17T18:55:43.018Z",
      "updated": "2017-04-17T18:55:43.018Z"
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/0247c2f5-4155-4be5-bd51-167de32f2f39/intents/order_pizza/examples?version=2016-09-20"
  }
}


In [27]:
response = conversation.update_example(workspace_id=workspace_id,
                                       intent='order_pizza',
                                       text='I want to order a pizza.',
                                       new_text='I want to order a pizza with pepperoni.')
print(json.dumps(response, indent=2))

{
  "text": "I want to order a pizza with pepperoni.",
  "created": "2017-04-17T18:55:42.172Z",
  "updated": "2017-04-17T18:58:00.647Z"
}


### Counterexamples

In [28]:
response = conversation.create_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice today.')
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice today.",
  "created": "2017-04-17T18:58:03.340Z",
  "updated": "2017-04-17T18:58:03.340Z"
}


In [29]:
response = conversation.get_counterexample(workspace_id=workspace_id,
                                           text='I want financial advice today.')
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice today.",
  "created": "2017-04-17T18:58:03.340Z",
  "updated": "2017-04-17T18:58:03.340Z"
}


In [30]:
response = conversation.list_counterexamples(workspace_id=workspace_id)
print(json.dumps(response, indent=2))

{
  "counterexamples": [
    {
      "text": "I want financial advice today.",
      "created": "2017-04-17T18:58:03.340Z",
      "updated": "2017-04-17T18:58:03.340Z"
    }
  ],
  "pagination": {
    "refresh_url": "/v1/workspaces/0247c2f5-4155-4be5-bd51-167de32f2f39/counterexamples?version=2016-09-20"
  }
}


In [31]:
response = conversation.update_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice today.',
                                              new_text='I want financial advice for tomorrow.')
print(json.dumps(response, indent=2))

{
  "text": "I want financial advice for tomorrow.",
  "created": "2017-04-17T18:58:03.340Z",
  "updated": "2017-04-17T18:58:09.403Z"
}


In [32]:
response = conversation.delete_counterexample(workspace_id=workspace_id,
                                              text='I want financial advice for tomorrow.')
print(json.dumps(response, indent=2))

{}


## Cleanup (Delete Pizza Chatbot)

Let's cleanup by deleting the pizza chatbot, since it is no longer needed.

In [33]:
# clean-up by deleting the workspace
conversation.delete_workspace(workspace_id=workspace_id)

{}